# Stock Screener

### This notebook is intended to help get a centralized view of important statistics for stocks of companies.

### 1. Imports
We begin with all the imports we will need for analysis.

In [2]:
import pickle
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup

### 2. Getting tickers for gold stocks.

In [11]:
#Where tickers comes from
goldStocksUrl = "http://www.miningfeeds.com/gold-mining-report-all-countries"

def goldStockParser(url, tickerColumn, stockTypes):
    """Goes to url above and pulls the tickers of each 
       gold mining company, dumps them in a pickle file 
       and returns the tickers as a list."""
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "lxml")
    table = soup.find("table")
    tickers = []
    for row in table.findAll("tr")[1:]:
        ticker = row.findAll("td")[tickerColumn].text
        #Randgold real ticker.
        if (ticker == "GOLD.L"):
            ticker = "GOLD"
        tickers.append(ticker)

    with open(stockTypes + ".pickle", "wb") as file:
        pickle.dump(tickers, file)

    return tickers

def pickleLoader(stockTypes):
    """Loads tickers from a pickle file and returns them as a list."""
    with open(stockTypes + ".pickle", "rb") as file:
        return pickle.load(file)

#Look in cache if False, go to website if true.    
reloadGoldTickers = False

badDataStocks = ["GLF.AX", "AZ.TO"]
goldStockTickers = []

if reloadGoldTickers:
    goldStockTickers = goldStockParser(goldStocksUrl, 2, "goldStocks")
else:
    goldStockTickers = pickleLoader("goldStocks")

#Remove stocks with bad data.    
for ticker in badDataStocks:
    goldStockTickers.remove(ticker)
    
print(goldStockTickers)

['NEM', 'ABX', 'FNV.TO', 'NCM.AX', 'GG', 'AEM', 'KGC', 'RGLD', 'RRS.L', 'FRES.L', 'AGG.AX', 'EVN.AX', 'GFI', 'BTO.TO', 'YRI.TO', 'IMG.TO', 'AGI.TO', 'CEY.L', 'PVG', 'DGC.TO', 'NGD.TO', 'RRL.AX', 'OGC.TO', 'SBM.AX', 'CG.TO', 'EDV.TO', 'NG.TO', 'ELD.TO', 'SAR.AX', 'SMF.TO', 'HMY', 'TXG.TO', 'CGG.TO', 'NSU.TO', 'GUY.TO', 'RSG.AX', 'SEA.TO', 'MUX', 'RIC', 'PG.TO', 'OSK.TO', 'ALIAF', 'CNL.TO', 'GOR.AX', 'POG.L', 'DNA.TO', 'GSS', 'PAF.L', 'AR.TO', 'PRU.AX', 'TBR.AX', 'BGM.V', 'TGZ.TO', 'SBB.TO', 'PRH.AX', 'GORO', 'WDO.TO', 'HRT.TO', 'WAF.AX', 'VIT.V', 'AOT.V', 'NGQ.TO', 'BDR.AX', 'HUM.L', 'RMS.AX', 'SLR.AX', 'BSX.TO', 'GCY.AX', 'KOR.TO', 'MOZ.TO', 'GBU.TO', 'BTR.V', 'HRR.AX', 'PRB.V', 'ALK.AX', 'HAS.AX', 'R.TO', 'RPM.V', 'ARZ.TO', 'JAG.TO', 'MAX.TO', 'TNG.AX', 'PEN.AX', 'CAL.TO', 'TLG.AX', 'RUP.V', 'GGP.L', 'AAL.V', 'KCN.AX', 'ITH.TO', 'MML.AX', 'GGG.AX', 'VGZ', 'ARU.V', 'RMX.TO', 'QBL.AX', 'TSG.L', 'ATC.V', 'KGL.AX', 'TML.TO', 'IMA.AX', 'DRM.AX', 'RED.AX', 'MTO.V', 'DGR.AX', 'MRC.AX', 'DNG.

### 3. Now collect data for each ticker

Specify rows of data that are wanted from yahoo.

Specify rows of data that have letters in their numbers for conversion.

Specify rows of data that have percentage signs in their numbers for conversion.

In [29]:
wantedRows = {0  : "marketCap", 
              1  : "enterpriseValue",
              2  : "trailingPe",
              3  : "forwardPe",
              4  : "5YrPeg",
              5  : "priceSales",
              6  : "priceBook",
              7  : "evSales",
              8  : "evEBITDA",
              11 : "profitMargin",
              12 : "operatingMargin",
              13 : "returnOnAssets",
              14 : "returnOnEquity",
              15 : "revenue",
              16 : "revenuePerShare",
              17 : "quarterlyRevenueGrowth",
              18 : "grossProfit",
              19 : "EBITDA",
              20 : "netIncomeForCommon",
              21 : "Diluted EPS",
              22 : "quarterlyEarningsGrowth",
              23 : "totalCash",
              24 : "cashPerShare",
              25 : "totalDebt",
              26 : "debtEquity",
              27 : "currentRatio",
              28 : "bookValuePerShare", 
              29 : "operatingCashFlow",
              30 : "LeveredFreeCashFlow",
              31 : "beta3YrMonthly",
              32 : "52WeekChange",
              33 : "spy52WeekChange",
              34 : "52WeekHigh",
              35 : "52WeekLow",
              36 : "50dayMvAvg",
              37 : "200dayMvAvg",
              38 : "avg3MonthVol",
              39 : "avg10DayVol",
              40 : "sharesOutstanding",
              41 : "sharesFloating",
              42 : "heldByInsiders",
              43 : "heldByInstitutions",
              44 : "sharesShort1MonthPrior",
              45 : "shortRatio",
              46 : "shortToFloat",
              47 : "shortToOutstanding",
              48 : "sharesShort2MonthsPrior",
              49 : "forwardDividendRate",
              50 : "forwardDividendYield",
              51 : "trailingDividendRate",
              52 : "trailingDividendYield",
              53 : "5yrAvgDividendYield",
              54 : "payoutRatio",
              55 : "dividendDate",
              56 : "exDividendDate",
              57 : "lastSplitFactorNewPerOld",
              58 : "lastSplitDate",
              60 : "currentPrice",}

We need to look up data from yahoo and store it in a csv locally for caching purposes. Also want to format well. ie remove redundant columns and remove symbols such as B and %.

In [33]:
baseurl = "https://uk.finance.yahoo.com/quote/"

def getDataFromYahoo(stockTypes, tickers, baseurl = baseurl):
    """Checks if data has been stored in csv, if not
       looks to yahoo. Then returns as map of tickers to dataframe."""
    #Create directory for data of tickers.
    dataDir = "./%s_data" % stockTypes
    createDirIfItDoesntExist(dataDir)
    
    frames = {}
    
    for ticker in tickers:
        dataFile = "./%s/%s.csv" % (dataDir, ticker)
        data = None
        if not os.path.exists(dataFile):
            data = saveDataAndReturnSeries(dataFile, ticker)
        else:
            print("Already have %s" % ticker)
            data = pd.read_csv(dataFile, index_col=0)
        frames[ticker] = data
        with pd.option_context('display.max_rows', 100, 'display.max_columns', 100):
            print(data)
    return frames

def createDirIfItDoesntExist(dataDir):
    """If directory with name dataDir does not exist, create it."""
    if not os.path.exists(dataDir):
        os.makedirs(dataDir)
        
def reformatData(data):
    """Reformat data with better names and remove redundancies"""
    dataDict = {}
    for index in wantedRows:
        series = data.loc[index]
        key = wantedRows[index]
        value = series.get(1)
        dataDict[key] = value
        dataDict["ticker"] = ticker
    return pd.Series(dataDict)    
        
def saveDataAndReturnSeries(dataFile, ticker):
    """Gets data from yahoo and dumps in csv file before 
       returning data as pandas series"""
    print("Requesting: %s" % ticker)

    #Go to summary page and scrape open.
    priceData = requests.get(baseurl + "%s?p=%s" % (ticker, ticker))
    summaryFrames = pd.read_html(priceData.text)
    summaryFrame = pd.concat(summaryFrames, ignore_index=True)
    priceFrame = summaryFrame.loc[1:1]

    #Go to stats page and scrape stats.
    data = requests.get(baseurl + "%s/key-statistics?p=%s" % (ticker, ticker))
    frames = pd.read_html(data.text)
    if len(frames) < 3:
        print("No data for: %s" % ticker)
        return

    #Join stats and price
    frames.append(priceFrame)
    frame = pd.concat(frames, ignore_index=True)
    
    
    reformattedSeries = reformatData(frame)
    print(reformattedSeries)
    
    #Save data
    reformattedSeries.to_csv(dataFile)
    return reformattedSeries

goldTickerData = getDataFromYahoo("gold_stocks", goldStockTickers)

Already have NEM
                               17.23B
marketCap                            
ticker                          AZ.TO
enterpriseValue                19.38B
trailingPe                        NaN
forwardPe                       23.78
5YrPeg                           6.64
priceSales                       2.42
priceBook                        1.63
evSales                          2.73
evEBITDA                         7.17
profitMargin                   -2.63%
operatingMargin                16.01%
returnOnAssets                  3.44%
returnOnEquity                 -1.62%
revenue                         7.11B
revenuePerShare                 13.34
quarterlyRevenueGrowth         -8.10%
grossProfit                     3.33B
EBITDA                           2.7B
netIncomeForCommon              -250M
Diluted EPS                     -0.35
quarterlyEarningsGrowth           NaN
totalCash                       3.13B
cashPerShare                     5.87
totalDebt                       4

marketCap                         3.38B
ticker                            AZ.TO
enterpriseValue                   4.42B
trailingPe                        15.66
forwardPe                         22.58
5YrPeg                            -3.61
priceSales                         1.04
priceBook                          0.74
evSales                            1.36
evEBITDA                           3.99
profitMargin                      6.85%
operatingMargin                   0.46%
returnOnAssets                    0.12%
returnOnEquity                    4.88%
revenue                           3.24B
revenuePerShare                    2.59
quarterlyRevenueGrowth           -8.90%
grossProfit                       1.55B
EBITDA                            1.11B
netIncomeForCommon               221.7M
Diluted EPS                        0.17
quarterlyEarningsGrowth             NaN
totalCash                          566M
cashPerShare                       0.45
totalDebt                         1.73B


marketCap                         5.92B
ticker                            AZ.TO
enterpriseValue                   5.71B
trailingPe                        24.29
forwardPe                       1848.97
5YrPeg                             2.69
priceSales                         4.88
priceBook                        159.69
evSales                            4.70
evEBITDA                           9.80
profitMargin                     20.30%
operatingMargin                  31.10%
returnOnAssets                    5.59%
returnOnEquity                    7.20%
revenue                           1.21B
revenuePerShare                   12.87
quarterlyRevenueGrowth           -7.10%
grossProfit                     511.92M
EBITDA                          582.74M
netIncomeForCommon              246.38M
Diluted EPS                      258.10
quarterlyEarningsGrowth          26.10%
totalCash                       653.53M
cashPerShare                       6.92
totalDebt                         2.77M


marketCap                         5.46B
ticker                            AZ.TO
enterpriseValue                   2.96B
trailingPe                        63.02
forwardPe                          3.08
5YrPeg                             0.11
priceSales                         1.29
priceBook                          0.43
evSales                            0.70
evEBITDA                           2.22
profitMargin                      0.42%
operatingMargin                  13.83%
returnOnAssets                    5.32%
returnOnEquity                    1.40%
revenue                           4.24B
revenuePerShare                   10.18
quarterlyRevenueGrowth           -5.30%
grossProfit                        780M
EBITDA                            1.33B
netIncomeForCommon                  18M
Diluted EPS                        0.04
quarterlyEarningsGrowth             NaN
totalCash                          221M
cashPerShare                       0.54
totalDebt                         2.05B


marketCap                         2.37B
ticker                            AZ.TO
enterpriseValue                   3.95B
trailingPe                          NaN
forwardPe                           NaN
5YrPeg                              NaN
priceSales                         0.84
priceBook                          0.88
evSales                            1.41
evEBITDA                           3.21
profitMargin                    -15.54%
operatingMargin                  16.28%
returnOnAssets                    4.48%
returnOnEquity                  -14.29%
revenue                           2.81B
revenuePerShare                    3.42
quarterlyRevenueGrowth            3.50%
grossProfit                      594.7M
EBITDA                            1.23B
netIncomeForCommon              -452.1M
Diluted EPS                       -0.53
quarterlyEarningsGrowth             NaN
totalCash                        498.1M
cashPerShare                       0.61
totalDebt                         1.89B


KeyboardInterrupt: 